In [1]:
import pandas as pd
import numpy as np
import ast
from utils import *
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.utils import shuffle
from scipy.sparse import coo_matrix
from sklearn.model_selection import train_test_split

/Users/zhiji/anaconda/envs/Fintech/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/zhiji/anaconda/envs/Fintech/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/zhiji/anaconda/envs/Fintech/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/zhiji/anaconda/envs/Fintech/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/zhiji/anaconda/envs/Fintech/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(

# 1. Object definition

In [20]:
class gesture_recognition:
    ''' 
    The argument of object constructor is the path(filename of the file you stored your data).
    If you would like to get train and test seperatedly, please name your training csv file as 'train.csv',
    and testing csv as 'test.csv'. 
    If your would like to got the recognition result by random split, please name your file as 'all.csv'. 
    and use train_test() function. That will take care of spliting, training and evaluating seperately.
    '''
    def __init__(self, filename):
        assert type(filename) == str
        self.path = './%s/'%(filename)
        self.labels = ['c','o','l','u','m','b','i','a']
    def load_all(self):
        return pd.read_csv(self.path+'all.csv')
    def load_train(self):
        return pd.read_csv(self.path+'train.csv')
    def load_test(self):
        return pd.read_csv(self.path + 'test.csv')
    def get_train(self, sample_rate = 30):
        df = self.load_train()
        X, y = [],[]
        for i, c in enumerate(self.labels):
            df_l = df.loc[df['label (S)'] == c]
            X_l, y_l = self.get_xy(df_l, i, sample_rate)
            X += X_l
            y += y_l
        X_sparse = coo_matrix(X)
        X, X_sparse, y = shuffle(X, X_sparse, y, random_state=0)
        scaler = preprocessing.StandardScaler().fit(X)
        X_nomalized = scaler.transform(X)
        return np.array(X_nomalized),np.array(y)
    def get_all(self, sample_rate, shaffle = False):
        X, y = [], []
        df = self.load_all()
        for i, c in enumerate(self.labels):
            df_l = df.loc[df['label (S)'] == c]
            X_l, y_l = self.get_xy(df_l, i, sample_rate)
            X += X_l
            y += y_l
        if shuffle:
            X_sparse = coo_matrix(X)
            X, X_sparse, y = shuffle(X, X_sparse, y, random_state=0)
        scaler = preprocessing.StandardScaler().fit(X)
        X_nomalized = scaler.transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_nomalized , y, test_size=0.3, random_state=42)
        return X_train, X_test, y_train, y_test
    def train_test(self, model):
        X_train, X_test, y_train, y_test = self.get_all(30)
        model.fit(X_train, y_train)
        training_accuracy = accuracy_score(y_train, model.predict(X_train))
        testing_accuracy = accuracy_score(y_test, model.predict(X_test))
        print("Training accuracy is: ", training_accuracy)
        print("Testing accuracy is: ", testing_accuracy)
        d_train = {"Train prediction is:  ": model.predict(X_train),
             "Groundtruth is: ": y_train
            }
        d_test = {"Train prediction is:  ": model.predict(X_test),
             "Groundtruth is: ": y_test
            }
        df_train = pd.DataFrame(data=d_train)
        df_test = pd.DataFrame(data=d_test)
        return df_train, df_test
            
    def get_test(self, sample_rate = 30):
        df = self.load_test()
        X, y = [],[]
        for i, c in enumerate(self.labels):
            df_l = df.loc[df['label (S)'] == c]
            X_l, y_l = self.get_xy(df_l, i, sample_rate)
            X += X_l
            y += y_l
        X_sparse = coo_matrix(X)
        X, X_sparse, y = shuffle(X, X_sparse, y, random_state=0)
        scaler = preprocessing.StandardScaler().fit(X)
        X_nomalized = scaler.transform(X)
        return np.array(X_nomalized),np.array(y)
    def train(self, model = svm(), verbose = False):
        X, y = self.get_train()
        model.fit(X, y)
        if verbose:
            training_accuracy = accuracy_score(y, model.predict(X))
            print("Training accuracy is: ", training_accuracy)
            d = {"Prediction is:  ": model.predict(X),
                 "Groundtruth is: ": y
                }
            df = pd.DataFrame(data=d)
            print(df)
            return model, df
        return model
    def test(self, model = svm(), verbose = False):
        model, df_train = self.train( model = model, verbose = verbose)
        X, y = self.get_test()
        
        if verbose:
            testing_accuracy = accuracy_score(y, model.predict(X))
            print("Testing accuracy is: ", testing_accuracy)
            d = {"Prediction is:  ": model.predict(X),
                 "Groundtruth is: ": y
                }
            df = pd.DataFrame(data=d)
            
            return testing_accuracy, df
        return testing_accuracy
            
    def get_xy(self, df, label, n = 30):
        features, labels = [], []
        for i in df['content (M)']:
            x = []
            d_x = ast.literal_eval(i)['data']['M']['x']['L']
            for j in range(len(d_x)):
                x.append(int(d_x[j]['N']))
            x = sampling(x, n)
            y = []
            d_y = ast.literal_eval(i)['data']['M']['x']['L']
            for j in range(len(d_y)):
                y.append(int(d_y[j]['N']))
            y = sampling(y,n)
            features.append(x + y)
            labels.append(label)
        return features, labels

# 2. Example manual

In [21]:
# 1. Get gesture recognition object
r = gesture_recognition('data')
# Get training result and test result by random split
df_train, df_test = r.train_test(model = svm())


Training accuracy is:  0.8794326241134752
Testing accuracy is:  0.6721311475409836


In [23]:
# Get training result seperately
# train_model = r.train(model = svm())
train_model, df_train = r.train(model = svm(), verbose = True)
# Get training result seperately
# ac_test= r.test(model = svm())
ac_test, df_test = r.test(model = svm(), verbose = True)

Training accuracy is:  0.8762376237623762
     Prediction is:    Groundtruth is: 
0                   0                 0
1                   6                 6
2                   4                 4
3                   3                 3
4                   6                 6
5                   7                 7
6                   0                 0
7                   5                 5
8                   0                 0
9                   5                 5
10                  5                 2
11                  4                 4
12                  6                 6
13                  2                 5
14                  3                 3
15                  0                 0
16                  1                 1
17                  5                 5
18                  1                 1
19                  2                 2
20                  6                 6
21                  5                 5
22                  6                 